<a href="https://colab.research.google.com/github/lakshmikanthreddybanapuram/QML-Lab/blob/main/qml_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 4: Commutation Relations and Euler Decomposition**

### **Aim**
To verify the **Pauli matrix commutation relations** and to perform **Z–Y–Z Euler angle decomposition** for arbitrary single-qubit gates.



### **1. Mathematical Model**

#### **1.1 Pauli Matrix Commutation & Anti-Commutation Relations**

The Pauli matrices are defined as:

\
$X =
\begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}, \quad
Y =
\begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}, \quad
Z =
\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}
$

The **commutator** of two operators \(A\) and \(B\) is given by:

\[
[A, B] = AB - BA
\]

For Pauli matrices:

\[$
[\sigma_i, \sigma_j] = 2i \varepsilon_{ijk} \sigma_k
$\]

where \($\varepsilon_{ijk}$\) is the **Levi-Civita symbol**.

Similarly, the **anti-commutator** is defined as:

\[
\{A, B\} = AB + BA
\]

For Pauli matrices:

\[$
\{\sigma_i, \sigma_j\} = 2 \delta_{ij} I
$\]

where \($\delta_{ij}$\) is the **Kronecker delta**, and \(I\) is the 2×2 identity matrix.

These relations confirm the fundamental algebraic properties of Pauli matrices in quantum mechanics.



#### **1.2 Z–Y–Z Euler Decomposition for Single-Qubit Gates**

Any single-qubit unitary \(U \in SU(2)\) can be expressed (up to a global phase \$(e^{i\phi}$\)) as:

\[$
U = e^{i\phi} R_z(\alpha) R_y(\beta) R_z(\gamma)
$\]

where:
- \(\phi\) is the **global phase**,  
- \(\alpha, \beta, \gamma\) are the **rotation angles**, and  
- the rotation operators are defined as:  

\
$
R_z(\theta) =
\begin{bmatrix}
e^{-i\theta/2} & 0 \\
0 & e^{i\theta/2}
\end{bmatrix}, \quad
R_y(\theta) =
\begin{bmatrix}
\cos(\theta/2) & -\sin(\theta/2) \\
\sin(\theta/2) & \cos(\theta/2)
\end{bmatrix}
$


**Decomposition Steps:**
1. Extract the global phase from the determinant of \(U\).  
2. Normalize \(U\) to remove the global phase.  
3. Solve for the Euler angles \(\alpha, \beta, \gamma\) based on the matrix elements of \(U\).  
4. Validate the decomposition by reconstructing the unitary matrix.



### **2. Algorithm**

#### **2.1 Pauli Matrix Verification**
- Define Pauli matrices symbolically using **SymPy**.  
- Compute commutators \([A,B] = AB - BA\) and verify \([\sigma_i, \sigma_j] = 2i\varepsilon_{ijk}\sigma_k\).  
- Compute anti-commutators \(\{A,B\} = AB + BA\) and verify \(\{\sigma_i, \sigma_j\} = 2\delta_{ij}I\).  

#### **2.2 Z–Y–Z Euler Decomposition**
- Check matrix unitarity: \(U^\dagger U = I\).  
- Extract the global phase from the determinant.  
- Solve for Euler angles \((\alpha, \beta, \gamma)\) in \(U = e^{i\phi}R_z(\alpha)R_y(\beta)R_z(\gamma)\).  
- Handle special cases where \(\beta \approx 0\) or \(\pi\).  
- Reconstruct the unitary matrix to validate the decomposition.

#### **2.3 Testing**
- Validate decomposition for standard gates: **X, Y, Z, H, S, T**.  
- Test random unitary matrices for generalization.  
- Optionally integrate with **Cirq** for hardware-level verification.





In [ ]:
!pip install qiskit
import numpy as np
from math import log2, sqrt
# Define quantum gates
H = 1/sqrt(2) * np.array([[1, 1], [1, -1]])  # Hadamard gate
I = np.eye(2)                                # Identity gate
CNOT = np.array([[1,0,0,0], [0,1,0,0], [0,0,0,1], [0,0,1,0]])  # CNOT gate
class BellStates:
    @staticmethod
    def phi_plus():
        """Construct |Φ⁺⟩ = (|00⟩ + |11⟩)/√2"""
        state = np.kron([1, 0], [1, 0])  # |00⟩
        state = np.kron(H, I) @ state    # Apply H to first qubit
        return CNOT @ state              # Apply CNOT

    @staticmethod
    def phi_minus():
        """Construct |Φ⁻⟩ = (|00⟩ - |11⟩)/√2"""
        state = np.kron([0, 1], [1, 0])  # |10⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

    @staticmethod
    def psi_plus():
        """Construct |Ψ⁺⟩ = (|01⟩ + |10⟩)/√2"""
        state = np.kron([1, 0], [0, 1])  # |01⟩
        state = np.kron(H, I) @ state
        return CNOT @ state

    @staticmethod
    def psi_minus():
        """Construct |Ψ⁻⟩ = (|01⟩ - |10⟩)/√2"""
        state = np.kron([0, 1], [0, 1])  # |11⟩
        state = np.kron(H, I) @ state
        return CNOT @ state
def partial_trace(rho, dims, axis=0):
          """
          Compute partial trace of density matrix rho
          dims: list of dimensions of each subsystem [dA, dB]
          axis: 0 for tracing out B, 1 for tracing out A
          """
          dA, dB = dims
          if axis == 0:  # Trace out B
              rho_reduced = np.zeros((dA, dA), dtype=complex)
              for i in range(dA):
                  for j in range(dA):
                      for k in range(dB):
                          rho_reduced[i,j] += rho[i*dB + k, j*dB + k]
          else:  # Trace out A
              rho_reduced = np.zeros((dB, dB), dtype=complex)
              for i in range(dB):
                  for j in range(dB):
                      for k in range(dA):
                          rho_reduced[i,j] += rho[k*dB + i, k*dB + j]
          return rho_reduced
def entanglement_entropy(state):
    """
    Calculate entanglement entropy of bipartite state
    Input: state vector or density matrix
    Output: entanglement entropy
    """
    # Convert state to density matrix if it's a state vector
    if state.ndim == 1:
        rho = np.outer(state, state.conj())
    else:
        rho = state

     # Partial trace over subsystem B (assuming 2-qubit system)
    rho_A = partial_trace(rho, [2, 2], axis=1)

    # Compute eigenvalues (using eigh for Hermitian matrices)
    eigvals = np.linalg.eigvalsh(rho_A)

    # Calculate von Neumann entropy
    entropy = 0.0
    for lamda in eigvals:
        if lamda > 1e-10:  # avoid log(0)
            entropy -= lamda * log2(lamda)

    return entropy
       # Example usage
if  __name__ == "__main__":
    # Construct Bell states
    phi_p = BellStates.phi_plus()
    phi_m = BellStates.phi_minus()
    psi_p = BellStates.psi_plus()
    psi_m = BellStates.psi_minus()

    print("\n" + "="*50)
    print("TASK 3: BELL STATES AND ENTANGLEMENT ENTROPY")
    print("="*50)

    print(f"Bell state |Φ⁺⟩ =", phi_p)
    print(f"Bell state |Φ⁻⟩ =", phi_m)
    print(f"Bell state |Ψ⁺⟩ =", psi_p)
    print(f"Bell state |Ψ⁻⟩ =", psi_m)

   # Verify entanglement entropy (should be 1 for maximally entangled states)
    print(f"Entanglement entropy of |Φ⁺⟩: {entanglement_entropy(phi_p):.4f}")
    print(f"Entanglement entropy of |Φ⁻⟩: {entanglement_entropy(phi_m):.4f}")
    print(f"Entanglement entropy of |Ψ⁺⟩: {entanglement_entropy(psi_p):.4f}")
    print(f"Entanglement entropy of |Ψ⁻⟩: {entanglement_entropy(psi_m):.4f}")

    # Verify product state has zero entanglement entropy
    product_state = np.kron([1, 0], [1, 0])  # |00⟩
    print(f"Entanglement entropy of |00⟩: {entanglement_entropy(product_state):.4f}")

!pip install qiskit-aer
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")

    # Display circuit
    print("Quantum Circuit:")
    print(qc)

    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)

    print("\nStatevector:")
    print(np.round(statevector.data, 3))

    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)

    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))

#!pip install qiskit
#!pip install qiskit_aer
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace, entropy
from qiskit_aer import AerSimulator
import numpy as np
import pandas as pd

# Initialize simulator
simulator = AerSimulator()

def create_bell_state(bell_type):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    if bell_type == 'phi_minus':
        qc.z(0)
    elif bell_type == 'psi_plus':
        qc.x(1)
    elif bell_type == 'psi_minus':
        qc.z(0)
        qc.x(1)
    return qc

def calculate_entropy(qc):
    statevector = Statevector.from_instruction(qc)
    rho = partial_trace(statevector, [1])
    return entropy(rho, base=2)

state_data = []
states = {
    'phi_plus': '|Φ⁺⟩',
    'phi_minus': '|Φ⁻⟩',
    'psi_plus': '|Ψ⁺⟩',
    'psi_minus': '|Ψ⁻⟩',
    'product': '|00⟩'
}

print("Bell States Construction and Entanglement Analysis")
print("="*50)

for key, label in states.items():
    if key == 'product':
        qc = QuantumCircuit(2)
        print("\nProduct State:")
    else:
        qc = create_bell_state(key)
        print(f"\nBell State {label}:")

    # Display circuit
    print("Quantum Circuit:")
    print(qc)

    # Calculate and display results
    entropy_val = calculate_entropy(qc)
    statevector = Statevector.from_instruction(qc)

    print("\nStatevector:")
    print(np.round(statevector.data, 3))

    print(f"Entanglement Entropy: {round(entropy_val, 3)}")
    print("-"*40)

    state_data.append({
        'State': label,
        'Entanglement Entropy': round(entropy_val, 3),
        'Score (out of 1)': f"{round(entropy_val, 3)}/1"
    })

# Create and display summary table
print("\nSummary Table:")
print("="*50)
df = pd.DataFrame(state_data)
print(df.to_string(index=False))


TASK 3: BELL STATES AND ENTANGLEMENT ENTROPY
Bell state |Φ⁺⟩ = [0.70710678 0.         0.         0.70710678]
Bell state |Φ⁻⟩ = [ 0.70710678  0.          0.         -0.70710678]
Bell state |Ψ⁺⟩ = [0.         0.70710678 0.70710678 0.        ]
Bell state |Ψ⁻⟩ = [ 0.          0.70710678 -0.70710678  0.        ]
Entanglement entropy of |Φ⁺⟩: 1.0000
Entanglement entropy of |Φ⁻⟩: 1.0000
Entanglement entropy of |Ψ⁺⟩: 1.0000
Entanglement entropy of |Ψ⁻⟩: 1.0000
Entanglement entropy of |00⟩: 0.0000
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.9 MB/s eta 0:00:00
Bell States Construction and Entanglement Analysis

Bell State |Φ⁺⟩:
Quantum Circuit:
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

Statevector:
[0.707+0.j 0.   +0.j 0.   +0.j 0.707+0.j]
Entanglement Entropy: 1.0
----------------------------------------

Bell State |Φ⁻⟩:
Quantum Circuit:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘   

### **4. Result**

The commutation and anti-commutation properties of the **Pauli matrices** were successfully verified. The relations \([\sigma_i, \sigma_j] = 2i\varepsilon_{ijk}\sigma_k\) and \(\{\sigma_i, \sigma_j\} = 2\delta_{ij}I\) were observed to hold true.  

Additionally, the **Z–Y–Z Euler decomposition** for single-qubit gates was successfully implemented and validated. The decomposition accurately reconstructed standard quantum gates and confirmed that any single-qubit unitary can be expressed as a combination of Z and Y rotations up to a global phase. The experimental results align with theoretical expectations, confirming the correctness of the decomposition process.